In [1]:
import geopy.distance
import requests
import pandas as pd
import numpy as np

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude value
import geopy.distance

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML     
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
! pip install folium==0.5.0
import folium # plotting library
#print('Folium installed')
#print('Libraries imported.')

In [2]:
address = '2041 Georgia Ave NW, Washington, DC 20060'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.91783385 -77.0201005008513


In [3]:
dc_neighborhoods = pd.read_csv('Downloads/DC_Neighborhood_Labels.csv')
dc_neighborhoods = dc_neighborhoods[['X','Y','NAME']].rename(columns={'X':'Longitude','Y':'Latitude','NAME':'Neighborhood'})
dc_neighborhoods.head()

,Longitude,Latitude,Neighborhood
0,-77.040877,38.991765,North Portal Estates
1,-77.041094,38.986790,Colonial Village
2,-77.032126,38.982980,Shepherd Park
3,-77.055934,38.980883,Hawthorne
4,-77.021558,38.976462,Takoma


In [5]:
def getNearbyVenues(names, lats, lngs, radius=2000):
    venues_list=[]
    
    for name, lat, lng in zip(names, lats, lngs):
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
LIMIT = 300
venues = getNearbyVenues(names=dc_neighborhoods['Neighborhood'],lats=dc_neighborhoods['Latitude'],lngs=dc_neighborhoods['Longitude'])
print(venues.shape)
venues.head()

(11127, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Portal Estates,38.991765,-77.040877,Parkway Deli & Restaurant,38.992585,-77.049176,Diner
1,North Portal Estates,38.991765,-77.040877,Peet's Coffee & Tea,38.992911,-77.032106,Coffee Shop
2,North Portal Estates,38.991765,-77.040877,The Emerald Door,38.992485,-77.048997,Spa
3,North Portal Estates,38.991765,-77.040877,District Taco,38.992744,-77.032405,Mexican Restaurant
4,North Portal Estates,38.991765,-77.040877,"The Daily Dish, A Restaurant & Catering Company",38.992316,-77.048954,American Restaurant


In [7]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 317 uniques categories.


In [8]:
venues['Venue Category'].unique()

array(['Diner', 'Coffee Shop', 'Spa', 'Mexican Restaurant',
       'American Restaurant', 'Brewery', 'Beer Store',
       'Indie Movie Theater', 'Dive Bar', 'Cuban Restaurant', 'Wine Shop',
       'Music Venue', 'Gym', 'Sushi Restaurant', 'Asian Restaurant',
       'Portuguese Restaurant', 'Salad Place', 'Train Station',
       'South American Restaurant', 'Trail', 'Fish Market',
       'Spanish Restaurant', 'Mediterranean Restaurant', 'Ice Cream Shop',
       'Grocery Store', 'Park', 'Thai Restaurant', 'Greek Restaurant',
       'Japanese Restaurant', 'Caribbean Restaurant', 'Plaza',
       'Record Shop', 'Food Truck', 'Fast Food Restaurant',
       'Arepa Restaurant', 'Vietnamese Restaurant', 'Movie Theater',
       'Liquor Store', 'Department Store', 'Pizza Place',
       'Ethiopian Restaurant', 'Bagel Shop', 'Indian Restaurant',
       'Pet Store', 'Dog Run', 'Seafood Restaurant', 'Burger Joint',
       'Bakery', 'Skating Rink', 'Food & Drink Shop', 'Bubble Tea Shop',
       'Gymna

In [9]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Zoo Exhibit,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
venues_grouped = venues_onehot.groupby('Neighborhood').sum().reset_index()
venues_grouped.head()

,Neighborhood,Zoo Exhibit,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,Art Museum,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,16th Street Heights,0,0,0,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Adams Morgan,3,1,0,2,0,0,0,1,1,...,0,2,1,0,0,0,0,0,1,0
2,American University Park,0,0,0,5,0,0,0,0,1,...,0,0,0,1,0,0,0,0,2,0
3,Arboretum,0,0,1,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,Barnaby Woods,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# coffee shops/bakeries in each
total_coffee=venues_grouped['Coffee Shop']+venues_grouped['Bakery']+venues_grouped['Pie Shop']+\
venues_grouped['Cupcake Shop']+venues_grouped['Dessert Shop']+venues_grouped['Café']+venues_grouped['Bagel Shop']

In [12]:
#activities
total_activities = venues_grouped['Indie Movie Theater']+venues_grouped['Music Venue']+venues_grouped['Performing Arts Venue']+venues_grouped['Theater']

In [13]:
#gyms
total_gyms = venues_grouped['Gym']+venues_grouped['Cycle Studio']+venues_grouped['Gym / Fitness Center']+\
venues_grouped['Outdoor Gym']+venues_grouped['Roller Rink']

In [14]:
#dog options
total_dog = venues_grouped['Park']+venues_grouped['Dog Run']
#+venues_grouped['State / Provincial Park']

In [15]:
#total number of restaurants in each 
total_restaurants = venues_grouped['African Restaurant']+venues_grouped['Arepa Restaurant']+\
venues_grouped['Asian Restaurant']+\
venues_grouped['Belgian Restaurant']+venues_grouped['Brazilian Restaurant']+\
venues_grouped['Cajun / Creole Restaurant']+venues_grouped['Cantonese Restaurant']+venues_grouped['Caribbean Restaurant']+\
venues_grouped['Chinese Restaurant']+venues_grouped['Comfort Food Restaurant']+\
venues_grouped['Cuban Restaurant']+venues_grouped['Dumpling Restaurant']+\
venues_grouped['Eastern European Restaurant']+venues_grouped['Ethiopian Restaurant']+\
venues_grouped['Falafel Restaurant']+venues_grouped['Filipino Restaurant']+\
venues_grouped['Fish & Chips Shop']+venues_grouped['French Restaurant']+venues_grouped['Fried Chicken Joint']+\
venues_grouped['German Restaurant']+venues_grouped['Greek Restaurant']+\
venues_grouped['Italian Restaurant']+\
venues_grouped['Japanese Restaurant']+venues_grouped['Korean Restaurant']+\
venues_grouped['Latin American Restaurant']+venues_grouped['Mediterranean Restaurant']+\
venues_grouped['Mexican Restaurant']+venues_grouped['Middle Eastern Restaurant']+venues_grouped['Pakistani Restaurant']+\
venues_grouped['Persian Restaurant']+venues_grouped['Peruvian Restaurant']+\
venues_grouped['Portuguese Restaurant']+venues_grouped['Ramen Restaurant']+\
venues_grouped['Scandinavian Restaurant']+venues_grouped['Seafood Restaurant']+\
venues_grouped['South American Restaurant']+venues_grouped['Spanish Restaurant']+\
venues_grouped['Steakhouse']+venues_grouped['Sushi Restaurant']+\
venues_grouped['Taco Place']+venues_grouped['Tapas Restaurant']+venues_grouped['Thai Restaurant']+\
venues_grouped['Turkish Restaurant']+venues_grouped['Vietnamese Restaurant']

In [16]:
df_interest = pd.DataFrame(data={'Neighborhood': venues_grouped['Neighborhood'],'Total Restaurants':total_restaurants,
                                  'Coffee Options':total_coffee,'Gym Options':total_gyms,'Dog_outdoor Options':total_dog,
                                'Activity Options':total_activities})
df_interest

,Neighborhood,Total Restaurants,Coffee Options,Gym Options,Dog_outdoor Options,Activity Options
0,16th Street Heights,20,7,0,3,1
1,Adams Morgan,23,10,3,3,1
2,American University Park,18,12,4,4,0
3,Arboretum,8,4,1,2,0
4,Barnaby Woods,3,2,0,4,1
...,...,...,...,...,...,...
127,West End,23,13,1,3,1
128,Woodland,9,3,3,2,2
129,Woodland-Normanstone,15,10,2,6,0
130,Woodley Park,23,10,1,4,0


In [18]:
df_interest.describe()

,Total Restaurants,Coffee Options,Gym Options,Dog_outdoor Options,Activity Options
count,132.000000,132.000000,132.000000,132.000000,132.000000
mean,14.878788,5.969697,1.984848,2.969697,1.265152
std,7.479264,4.262278,1.533269,1.929836,1.517631
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,2.000000,1.000000,2.000000,0.000000
50%,15.000000,7.000000,2.000000,3.000000,1.000000
75%,21.000000,10.000000,3.000000,4.000000,2.000000
max,31.000000,16.000000,6.000000,9.000000,7.000000


In [19]:
df_interest = df_interest.loc[(df_interest['Total Restaurants']>6)&(df_interest['Coffee Options']>2)&(df_interest['Gym Options']>2)&(df_interest['Dog_outdoor Options']>1)&(df_interest['Activity Options']>=1)]
#df_interest = df_interest.loc[(df_interest['Total Restaurants']>6)&(df_interest['Coffee Options']>2)&(df_interest['Gym Options']>1)&(df_interest['Dog_outdoor Options']>2)&(df_interest['Activity Options']>2)]
print(df_interest.shape)
df_interest

(24, 6)


,Neighborhood,Total Restaurants,Coffee Options,Gym Options,Dog_outdoor Options,Activity Options
1,Adams Morgan,23,10,3,3,1
16,Buzzard Point,14,7,6,5,5
17,Capitol Hill,20,8,3,6,1
20,Carver,19,7,3,2,3
23,Chevy Chase,19,8,3,3,1
26,Colonial Village,30,7,4,4,3
35,Dupont Circle,23,10,4,3,2
39,Edgewood,17,9,3,3,2
41,Fairlawn,11,5,3,3,1
42,Foggy Bottom,20,13,3,3,1


In [20]:
#merge neighborhood's of interest with latitude/longitude info
neighborhoods_interest = df_interest.join(dc_neighborhoods.set_index('Neighborhood'), on='Neighborhood')
work_coords=(latitude,longitude)
def neighborhood_dist(row):
    neighborhood_coords=(row['Latitude'],row['Longitude'])
    return geopy.distance.distance(work_coords,neighborhood_coords).miles
neighborhoods_interest['Distance'] = neighborhoods_interest.apply(lambda row: neighborhood_dist(row),axis=1)
print(neighborhoods_interest.shape)
neighborhoods_interest.head()

(24, 9)


,Neighborhood,Total Restaurants,Coffee Options,Gym Options,Dog_outdoor Options,Activity Options,Longitude,Latitude,Distance
1,Adams Morgan,23,10,3,3,1,-77.042391,38.920472,1.214913
16,Buzzard Point,14,7,6,5,5,-77.011425,38.868300,3.448726
17,Capitol Hill,20,8,3,6,1,-76.998477,38.883621,2.632157
20,Carver,19,7,3,2,3,-76.977850,38.902465,2.511796
23,Chevy Chase,19,8,3,3,1,-77.066478,38.963146,4.001535


In [21]:
close_neighborhoods = neighborhoods_interest.loc[neighborhoods_interest.Distance<=1.5]
close = close_neighborhoods.Neighborhood.unique
print(close_neighborhoods.shape)
close_neighborhoods

(5, 9)


,Neighborhood,Total Restaurants,Coffee Options,Gym Options,Dog_outdoor Options,Activity Options,Longitude,Latitude,Distance
1,Adams Morgan,23,10,3,3,1,-77.042391,38.920472,1.214913
35,Dupont Circle,23,10,4,3,2,-77.040984,38.912128,1.192265
39,Edgewood,17,9,3,3,2,-77.000739,38.925199,1.160433
66,Howard University,21,9,3,2,1,-77.019610,38.922355,0.312977
117,Stronghold,20,8,3,2,2,-77.007767,38.925776,0.861272


In [22]:
#map locations that meet requirements
neighborhood_map = folium.Map(location=[latitude,longitude],zoom_start=13)

#add marker for new job location
folium.CircleMarker(
[latitude,longitude],
radius=10,
color='red',
popup='Office',
fill=True,
fill_color='red',
fill_opacity=0.6).add_to(neighborhood_map)

#add markers for the preferred neighborhoods
for lat, lng, label in zip(close_neighborhoods.Latitude, close_neighborhoods.Longitude, close_neighborhoods.Neighborhood):
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    color='blue',
    popup=label,
    fill=True,
    fill_color='blue',
    fill_opacity=0.6).add_to(neighborhood_map)
    

In [23]:
neighborhood_map

In [28]:
specific_restaurants=venues_grouped.copy()
specific_restaurants=specific_restaurants[specific_restaurants.Neighborhood.isin(['Adams Morgan','Dupont Circle','Edgewood','Howard University','Stronghold'])][['Neighborhood','African Restaurant',
                'Arepa Restaurant','Asian Restaurant','Belgian Restaurant','Brazilian Restaurant',
               'Cajun / Creole Restaurant','Cantonese Restaurant','Caribbean Restaurant','Chinese Restaurant','Comfort Food Restaurant',
               'Cuban Restaurant','Dumpling Restaurant','Eastern European Restaurant','Ethiopian Restaurant','Falafel Restaurant',
               'Filipino Restaurant','Fish & Chips Shop','French Restaurant','Fried Chicken Joint','German Restaurant',
               'Greek Restaurant','Italian Restaurant','Japanese Restaurant','Korean Restaurant','Latin American Restaurant',
               'Mediterranean Restaurant','Mexican Restaurant','Middle Eastern Restaurant','Pakistani Restaurant',
               'Persian Restaurant','Peruvian Restaurant','Portuguese Restaurant','Ramen Restaurant','Scandinavian Restaurant',
               'Seafood Restaurant','South American Restaurant','Spanish Restaurant','Steakhouse','Sushi Restaurant',
               'Taco Place','Tapas Restaurant','Thai Restaurant','Turkish Restaurant','Vietnamese Restaurant']].set_index('Neighborhood').transpose()

In [30]:
adams = specific_restaurants[(specific_restaurants['Adams Morgan']!=0)].shape[0]
dupont = specific_restaurants[(specific_restaurants['Dupont Circle']!=0)].shape[0]
edgewood = specific_restaurants[(specific_restaurants['Edgewood']!=0)].shape[0]
howard = specific_restaurants[(specific_restaurants['Howard University']!=0)].shape[0]
stronghold = specific_restaurants[(specific_restaurants['Stronghold']!=0)].shape[0]
print(f'There are {adams} unique cuisines in Adams Morgan,\nthere are {dupont} unique cuisines in Dupont Circle,\nthere are {edgewood} unique cuisines in Edgewood,\nthere are {howard} unique cuisines in Howard University, and\nthere are {stronghold} unique cuisines in Stronghold')

There are 17 unique cuisines in Adams Morgan,
there are 15 unique cuisines in Dupont Circle,
there are 14 unique cuisines in Edgewood,
there are 16 unique cuisines in Howard University, and
there are 14 unique cuisines in Stronghold


In [17]:
df_interest[df_interest.Neighborhood=='Howard University'].transpose()

,66
Neighborhood,Howard University
Total Restaurants,21
Coffee Options,9
Gym Options,3
Dog_outdoor Options,2
Activity Options,1
